# Premiers tests de requêtes en base

Quelques imports à faire avant toute chose

In [3]:
import os
from batid.models import Building
# necessary for SQL query execution from Jupyter
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

## Compter les bâtiments dans le RNB


In [4]:
entries_nb = Building.objects.count()
entries_nb

721294

## Récupérer un bâtiment à partir de son RNB-id

In [5]:
building = Building.objects.get(rnb_id='QDLS5TEESYGR')

In [6]:
print(building.point)

SRID=2154;POINT (897010.1000000003 6476168.9)


## Vérifier si un point se situe à l'intérieur d'un ou plusieurs bâtiments stockés dans le RNB

In [7]:
from django.contrib.gis.geos import Point
point = Point(897010.1000000003, 6476168.9, srid=2154)
buildings = list(Building.objects.filter(shape__contains=point))
print(f"ce point est situé dans {len(buildings)} bâtiment(s) trouvé(s) dans le RNB")

ce point est situé dans 1 bâtiment(s) trouvé(s) dans le RNB


Example de point en dehors de tout bâtiment

In [8]:
point = Point(890010.0, 6476168.9, srid=2154)
buildings = list(Building.objects.filter(shape__contains=point))
print(f"ce point est situé dans {len(buildings)} bâtiment(s) trouvé(s) dans le RNB")

ce point est situé dans 0 bâtiment(s) trouvé(s) dans le RNB


## Trouver tous les bâtiments à moins de X mètres d'un point

In [9]:
from django.contrib.gis.measure import D # D is shortcut for distance

point = Point(897010.1000000003, 6476168.9, srid=2154)
buildings = Building.objects.filter(shape__distance_lte=(point, 100))
print(f"{len(buildings)} bâtiment(s) trouvé(s) dans un rayon de 100m autour du point")

4 bâtiment(s) trouvé(s) dans un rayon de 100m autour du point


Les ordonner par distance croissante

In [10]:
from django.contrib.gis.db.models.functions import Distance

point = Point(897010.1000000003, 6476168.9, srid=2154)
buildings = Building.objects.filter(shape__dwithin=(point, 100)).annotate(distance=Distance('shape', point)).order_by('distance')
for building in buildings:
    print(f"{building.rnb_id} à {building.distance}m")

QDLS5TEESYGR à 0.0 mm
L3J12FEAKC5J à 10.700467280145181 mm
R3TC71BSLG1D à 70.98034939342104 mm
3L827UV22MDN à 88.99943820057138 mm


## Accéder à la requête en base sous jacente

In [11]:
query = Building.objects.filter(shape__dwithin=(point, 100)).annotate(distance=Distance('shape', point)).order_by('distance').query

print(query)

SELECT "batid_building"."id", "batid_building"."rnb_id", "batid_building"."source", "batid_building"."point"::bytea, "batid_building"."shape"::bytea, "batid_building"."ext_bdnb_id", "batid_building"."ext_bdtopo_id", "batid_building"."created_at", "batid_building"."updated_at", ST_Distance("batid_building"."shape", ST_GeomFromEWKB('\001\001\000\000 j\010\000\0006333\344_+A\232\231\2319b\264XA'::bytea)) AS "distance" FROM "batid_building" WHERE ST_DWithin("batid_building"."shape", ST_GeomFromEWKB('\001\001\000\000 j\010\000\0006333\344_+A\232\231\2319b\264XA'::bytea), 100) ORDER BY "distance" ASC


## Trouver un bâtiment à partir d'une adresse

In [43]:
import requests

def get_coordinates_best_result(results):
    if len(results) == 0:
        return None
    
    best_result = results[0]
    geom = best_result['geometry']
    if geom['type'] == 'Point' and 'coordinates' in geom:
        print(geom['coordinates'])
        [lon, lat] = geom['coordinates']
        return [lon, lat]
    else:
        return None

def geocode_address(address):
    url = f'https://api-adresse.data.gouv.fr/search/?q={address}'
    response = requests.get(url)
    data = response.json()
    results = data["features"]
    best_result = get_coordinates_best_result(results)    
    return best_result

def find_building_from_wgs_84_coordinates(lon, lat):
    point = Point(lon, lat, srid=4326)
    buildings = Building.objects.filter(shape__dwithin=(point, 100)).annotate(distance=Distance('shape', point)).order_by('distance')
    # print(buildings)
    print(len(buildings))
    return buildings[0]

address = '1 Rue des Eaux Claires, Grenoble'
[lat, lon] = geocode_address(address)
building = find_building_from_wgs_84_coordinates(lat, lon)

[5.711453, 45.178974]
46


In [52]:
import json
from ipyleaflet import Map, Marker, GeoJSON

def show_map(geocoding_coordinates, building):
    [lat, lon] = geocoding_coordinates
    shape = building.shape
    shape_4326 = shape.transform(4326)
    geojson = json.loads(shape.geojson)
    center = (lon, lat)
    m = Map(center=center, zoom=17)
    marker = Marker(location=center, draggable=False)
    m.add_layer(marker);
    geo_json = GeoJSON(data=geojson)
    m.add_layer(geo_json)
    display(m)

In [53]:
show_map([lat, lon], building)

Map(center=[45.178974, 5.711453], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …